In [1]:
from bert2head.model import BertCNN2HEAD, BertLinear2HEAD, BertCNN2HEAD_UIT
import torch



cuda


In [2]:
net = BertLinear2HEAD('vinai/phobert-base-v2')
net.load_state_dict(torch.load("models/linear/Epoch-34-2-head-linear-smart-7_3-v1.1.pt"))


Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<All keys matched successfully>

In [3]:
torch.save(net.BertModel.state_dict(),"models/finetuneBertsmart.pt")